# 1. Training Set 생성

In [1]:
import pandas as pd
import numpy as np


df = pd.DataFrame()
callrate = pd.read_csv('./data/market/callrate_lable.csv')

#bond reports
bond = pd.read_csv('./data/bondreports/bondreports_preprocessed.csv')
bond['date'] = '20'+ bond['date']
bond['date'] = pd.to_datetime(bond['date'],format='%Y.%m.%d', errors='ignore')
callrate['date'] = pd.to_datetime(callrate['date'],format='%Y.%m.%d', errors='ignore')

df = pd.merge(bond, callrate, on='date', how='left')
df = df.fillna('')
df['ngrams in doc'] = df["ngram"] + ',' +  df["tokens"] 
df[['date','ngrams in doc','lable']].to_csv('./data/model/training_set.csv', mode='w', index=False)


In [2]:
#minutes
minutes = pd.read_csv('./data/minutes/minutes_preprocessed.csv')
minutes['date'] = pd.to_datetime(minutes['date'],format='%Y%m%d', errors='ignore')
callrate['date'] = pd.to_datetime(callrate['date'],format='%Y.%m.%d', errors='ignore')

df = pd.DataFrame()
df = pd.merge(minutes.loc[minutes.date <= '2017.12.31'], callrate, on='date', how='left')
df = df.fillna('')
df['ngrams in doc'] = df["Foreign Currency tokens"] + ',' +  df["Financial Markets tokens"]  + ',' +  df["Foreign Currency ngram"]  + ',' +  df["Financial Markets ngram"] 
df[['date','ngrams in doc','lable']].to_csv('./data/model/training_set.csv', mode='a', header=False, index=False)



In [3]:
import pandas as pd
from os import listdir
from os.path import isfile, join

#news
source_folder = "./data/news/"
news_ls = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
for newsfile in news_ls :
    if newsfile in ["news_2005_preprocessed_n.csv","news_2006_preprocessed_n.csv","news_2007_preprocessed_n.csv","news_2008_preprocessed_n.csv","news_2009_preprocessed_n.csv","news_2010_preprocessed_n.csv","news_2011_preprocessed_n.csv","news_2012_preprocessed_n.csv","news_2013_preprocessed_n.csv","news_2014_preprocessed_n.csv","news_2015_preprocessed_n.csv"] :
        news = pd.read_csv(source_folder + newsfile)
        news['date'] = pd.to_datetime(news['date'],format='%Y.%m.%d', errors='ignore')
        callrate['date'] = pd.to_datetime(callrate['date'],format='%Y.%m.%d', errors='ignore')

        df = pd.DataFrame()
        df = pd.merge(news, callrate, on='date', how='left')
        df = df.fillna('')
        df['ngrams in doc'] = df["tokens"] + ',' +  df["ngram"] 
        df[['date','ngrams in doc','lable']].to_csv('./data/model/training_set.csv', header=False, mode='a', index=False)

In [4]:
traning_set = pd.read_csv('./data/model/training_set.csv')
prev = 0
for i, row in traning_set.sort_values(by=['date'], ascending=False).iterrows() :
    if row['lable'] is None :
        print(i)
    
    prev = row['lable']

In [5]:
traning_set.sort_values(by=['date'], ascending=False).head()

,date,ngrams in doc,lable
391,2017-12-27,"은행채/NNG;신용스프레드/NNG;확대/NNG,신용스프레드/NNG;하향/NNG;안정...",1.0
78,2017-12-15,"fed/NNG;시장/NNG;예상/NNG;금리/NNG;인상/NNG,실업률/NNG;하락...",1.0
148,2017-12-04,"소비자/NNG;물가/NNG;상승률/NNG;최저/NNG,가계/NNG;부채/NNG;금리...",0.0
719,2017-11-30,"금융경제/NNG,보/VV,fed/NNG,금리/NNG,인상/NNG,하/VV,예견/NN...",1.0
270,2017-11-30,"그럼에도/VA;불구/NNG;금리/NNG;인상/NNG,금리/NNG;인하/NNG;사이클...",1.0


In [6]:
traning_set.loc[traning_set.lable==1.0].shape

(100635, 3)

In [7]:
traning_set.loc[traning_set.lable==-1.0].shape

(95512, 3)